# Bluevia Sepsis Labeling Based on SIRS Criteria

This code implements a clinical phenotyping approach to identify patients with sepsis based on the **Systemic Inflammatory Response Syndrome (SIRS) criteria**.

## What is SIRS?

SIRS (Systemic Inflammatory Response Syndrome) is a clinical response to various insults. A patient meets SIRS criteria if they have **≥2 of the following 4 abnormalities**:

### SIRS Criteria:
- **Temperature**: >38°C or <36°C
- **Heart Rate**: >90 beats/min
- **Respiratory Rate**: >20 breaths/min (or PaCO2 <32 mmHg)
- **WBC Count**: >12,000/mm³, <4,000/mm³, or >10% immature (band) forms

## How the Code Works:

### 1. **Abnormality Detection Functions**
- `mark_abnormal_temperature()`: Flags temperature readings outside normal range
- `mark_abnormal_heart_rate()`: Flags elevated heart rates
- `mark_abnormal_respiratory_rate()`: Flags high respiratory rates OR low PaCO2
- `mark_abnormal_wbc()`: Flags abnormal white blood cell counts

### 2. **Multiple Abnormal Events Detection**
- `get_patients_with_multiple_abnormals_timeframe()`: Finds patients with **≥2 DIFFERENT types** of abnormalities within a specified timeframe (default 24 hours)

**Key requirement**: Must be different types (e.g., temperature + heart rate), not multiple instances of the same type.

### 3. **Patient Labeling**
- `label_patients_with_sepsis_from_abnormals()`: Adds sepsis labels to patients DataFrame

**Outputs added to patients table**:
- `bluevia_sepsis`: 1 if patient met SIRS criteria, 0 otherwise
- `sepsis_date`: Date of second abnormal event (onset of systemic response)
- `sepsis_event_types`: Types of abnormalities (e.g., "temp_high,hr_high")
- `sepsis_events_count`: Number of abnormal events in timeframe
- `sepsis_timeframe_hours`: Hours between first and second abnormal events

## Clinical Significance:

This approach identifies the **onset of systemic inflammatory response** when a patient experiences multiple different physiological abnormalities within a short timeframe. The date of the **second abnormal event** represents when the systemic response began, which is clinically meaningful for:

- **Early sepsis detection**: Before traditional clinical diagnosis
- **Phenotype definition**: Based on physiological response, not just diagnosis codes
- **Temporal precision**: Uses specific timeframe to capture acute deterioration

## Example Output:

In [1]:
import pandas as pd
import numpy as np

patients = pd.read_csv("data/raw/patients.csv")
conditions = pd.read_csv("data/raw/conditions.csv")
encounters = pd.read_csv("data/raw/encounters.csv")
observations = pd.read_csv("data/raw/observations.csv")
procedures = pd.read_csv("data/raw/procedures.csv")

def mark_abnormal_temperature(observations_df, temp_threshold_c_high=38.0, temp_threshold_c_low=36.0):
    """
    Search observations for temperature readings that meet criteria (>38°C or <36°C).
    """
    # Make a copy to avoid modifying original
    df = observations_df.copy()

    # Temperature-related keywords
    temp_keywords = ["temperature", "temp", "fever", "body temperature"]

    # Filter for temperature observations
    temp_mask = df['DESCRIPTION'].str.lower().str.contains(
        '|'.join(temp_keywords), na=False
    )

    # Convert VALUE to numeric (only for temp observations)
    df['VALUE_NUM'] = pd.to_numeric(df['VALUE'], errors='coerce')

    # Handle temperature unit conversion (only for temp observations)
    def normalize_temperature(row):
        if not temp_mask.loc[row.name]:  # Skip if not temperature observation
            return np.nan
        temp_value = row['VALUE_NUM']
        if pd.isna(temp_value):
            return np.nan
        units = str(row['UNITS']).lower()

        if 'f' in units or 'fahr' in units:
            return (temp_value - 32) * 5/9
        elif 'c' in units or 'cels' in units:
            return temp_value
        else:
            return temp_value

    df['TEMP_C'] = df.apply(normalize_temperature, axis=1)

    # Create abnormal temperature mask
    abnormal_temp_mask = (
        temp_mask &  # Must be a temperature observation
        df['TEMP_C'].notna() &  # Must have valid temperature
        (
            (df['TEMP_C'] > temp_threshold_c_high) |
            (df['TEMP_C'] < temp_threshold_c_low)
        )
    )

    # Mark abnormal temperatures
    df['abnormal_temperature'] = 0
    df.loc[abnormal_temp_mask, 'abnormal_temperature'] = 1

    return df

def mark_abnormal_heart_rate(observations_df, hr_threshold_high=90):
    """
    Find observations with abnormal heart rates (>90 beats/min).
    """
    # Make a copy to avoid modifying original
    df = observations_df.copy()

    # Comprehensive heart rate keywords
    hr_keywords = [
        "heart rate", "pulse", "hr", "bpm", "beats", "heart rate rhythm",
        "cardiac rate", "pulse rate", "heart beat", "heartbeat",
        "ventricular rate", "atrial rate", "radial pulse", "carotid pulse"
    ]

    # Filter for heart rate observations
    hr_mask = df['DESCRIPTION'].str.lower().str.contains(
        '|'.join(hr_keywords), na=False
    )

    # Convert VALUE to numeric (preserve existing VALUE_NUM if exists)
    if 'VALUE_NUM' not in df.columns:
        df['VALUE_NUM'] = pd.to_numeric(df['VALUE'], errors='coerce')

    # Handle heart rate unit conversion (only for HR observations)
    def normalize_heart_rate(row):
        if not hr_mask.loc[row.name]:  # Skip if not heart rate observation
            return np.nan
        hr_value = row['VALUE_NUM']
        if pd.isna(hr_value):
            return np.nan
        units = str(row['UNITS']).lower()

        if '/s' in units or 'per second' in units or 'bps' in units:
            return hr_value * 60
        elif 'hz' in units:
            return hr_value * 60
        else:
            return hr_value

    df['HR_BPM'] = df.apply(normalize_heart_rate, axis=1)

    # Create abnormal heart rate mask
    abnormal_hr_mask = (
        hr_mask &  # Must be a heart rate observation
        df['HR_BPM'].notna() &  # Must have valid heart rate
        (df['HR_BPM'] > hr_threshold_high)  # Must be above threshold
    )

    # Mark abnormal heart rates
    df['abnormal_heart_rate'] = 0
    df.loc[abnormal_hr_mask, 'abnormal_heart_rate'] = 1

    return df

def mark_abnormal_respiratory_rate(observations_df, rr_threshold_high=20, paco2_threshold_low=32):
    """
    Find observations with abnormal respiratory rates (>20 breaths/min or PaCO2 <32 mmHg).
    """
    # Make a copy to avoid modifying original
    df = observations_df.copy()

    # Comprehensive respiratory-related keywords
    resp_keywords = [
        "respiratory rate", "rr", "breaths", "respiration", "ventilation",
        "breathing rate", "respiratory frequency", "tidal volume",
        "minute ventilation", "paco2", "carbon dioxide", "co2",
        "partial pressure co2", "arterial co2", "blood gas"
    ]

    # Filter for respiratory observations
    resp_mask = df['DESCRIPTION'].str.lower().str.contains(
        '|'.join(resp_keywords), na=False
    )

    # Convert VALUE to numeric (preserve existing VALUE_NUM if exists)
    if 'VALUE_NUM' not in df.columns:
        df['VALUE_NUM'] = pd.to_numeric(df['VALUE'], errors='coerce')

    # Handle respiratory rate unit conversion (only for respiratory observations)
    def normalize_respiratory_rate(row):
        if not resp_mask.loc[row.name]:  # Skip if not respiratory observation
            return np.nan
        resp_value = row['VALUE_NUM']
        if pd.isna(resp_value):
            return np.nan
        units = str(row['UNITS']).lower()

        # Convert from breaths per second to breaths per minute
        if '/s' in units or 'per second' in units:
            return resp_value * 60
        # Convert from Hz to breaths per minute (1 Hz = 60 BPM equivalent)
        elif 'hz' in units:
            return resp_value * 60
        # Already in breaths per minute or unclear units
        else:
            return resp_value

    df['RR_BPM'] = df.apply(normalize_respiratory_rate, axis=1)

    # Handle PaCO2 values (separate from respiratory rate)
    def normalize_paco2(row):
        if not resp_mask.loc[row.name]:  # Skip if not respiratory observation
            return np.nan
        desc = str(row['DESCRIPTION']).lower()
        if 'paco2' not in desc and 'co2' not in desc:  # Not a CO2 measurement
            return np.nan
        paco2_value = row['VALUE_NUM']
        if pd.isna(paco2_value):
            return np.nan
        units = str(row['UNITS']).lower()

        # Convert PaCO2 to mmHg if needed
        if 'kpa' in units:
            return paco2_value * 7.50062  # kPa to mmHg conversion
        elif 'cmh2o' in units:
            return paco2_value * 0.73556  # cmH2O to mmHg conversion
        else:
            return paco2_value

    df['PACO2_MMHG'] = df.apply(normalize_paco2, axis=1)

    # Create abnormal respiratory mask
    abnormal_resp_mask = (
        resp_mask &  # Must be a respiratory observation
        df['RR_BPM'].notna() &  # Must have valid respiratory rate
        (df['RR_BPM'] > rr_threshold_high)  # Must be above threshold
    ) | (
        resp_mask &  # Must be a respiratory observation
        df['PACO2_MMHG'].notna() &  # Must have valid PaCO2
        (df['PACO2_MMHG'] < paco2_threshold_low)  # Must be below threshold
    )

    # Mark abnormal respiratory rates
    df['abnormal_respiratory_rate'] = 0
    df.loc[abnormal_resp_mask, 'abnormal_respiratory_rate'] = 1

    return df

def mark_abnormal_wbc(observations_df, wbc_high=12000, wbc_low=4000, immature_percent_high=10):
    """
    Find observations with abnormal WBC counts (>12,000/mm³, <4,000/mm³, or >10% immature forms).
    """
    # Make a copy to avoid modifying original
    df = observations_df.copy()

    # Comprehensive WBC-related keywords
    wbc_keywords = [
        "wbc", "white blood", "leukocyte", "leukocytes", "white blood cell",
        "band", "band form", "immature", "neutrophil", "granulocyte",
        "white cell", "blood count", "hematology", "cbc", "complete blood"
    ]

    # Filter for WBC observations
    wbc_mask = df['DESCRIPTION'].str.lower().str.contains(
        '|'.join(wbc_keywords), na=False
    )

    # Convert VALUE to numeric (preserve existing VALUE_NUM if exists)
    if 'VALUE_NUM' not in df.columns:
        df['VALUE_NUM'] = pd.to_numeric(df['VALUE'], errors='coerce')

    # Handle WBC count unit conversion (only for WBC observations)
    def normalize_wbc_count(row):
        if not wbc_mask.loc[row.name]:  # Skip if not WBC observation
            return np.nan
        wbc_value = row['VALUE_NUM']
        if pd.isna(wbc_value):
            return np.nan

        desc = str(row['DESCRIPTION']).lower()
        units = str(row['UNITS']).lower()

        # Skip if this is about percentages/immature forms, not count
        if 'percent' in desc or '%' in desc or 'immature' in desc or 'band' in desc:
            return np.nan

        # Convert WBC count to cells/mm³
        if 'k/ul' in units or 'thousand/ul' in units:
            return wbc_value * 1000  # K/μL to cells/mm³
        elif 'x10^9/l' in units or '10^9/l' in units:
            return wbc_value * 1000  # 10^9/L to cells/mm³
        elif 'g/l' in units:  # Some labs report WBC in g/L
            return wbc_value * 1000000  # g/L to cells/mm³ (approximate)
        elif '10^3/ul' in units or '10^3/μl' in units:
            return wbc_value * 1000  # 10^3/μL to cells/mm³
        else:
            return wbc_value  # Assume already in cells/mm³

    df['WBC_COUNT'] = df.apply(normalize_wbc_count, axis=1)

    # Handle immature forms percentage
    def normalize_immature_percent(row):
        if not wbc_mask.loc[row.name]:  # Skip if not WBC observation
            return np.nan
        desc = str(row['DESCRIPTION']).lower()
        units = str(row['UNITS']).lower()

        # Only process if this is about immature forms/bands
        if not any(keyword in desc for keyword in ['immature', 'band', 'band form', 'neutrophil band']):
            return np.nan

        immature_value = row['VALUE_NUM']
        if pd.isna(immature_value):
            return np.nan

        # Convert to percentage if needed
        if 'fraction' in units or 'decimal' in units:
            return immature_value * 100  # Convert to percentage
        elif 'ratio' in units:
            return immature_value * 100  # Convert ratio to percentage
        else:
            return immature_value  # Assume already in percentage

    df['IMMATURE_PERCENT'] = df.apply(normalize_immature_percent, axis=1)

    # Create abnormal WBC mask
    abnormal_wbc_mask = (
        wbc_mask &  # Must be a WBC observation
        df['WBC_COUNT'].notna() &  # Must have valid WBC count
        (
            (df['WBC_COUNT'] > wbc_high) |  # High WBC
            (df['WBC_COUNT'] < wbc_low)     # Low WBC
        )
    ) | (
        wbc_mask &  # Must be a WBC observation
        df['IMMATURE_PERCENT'].notna() &  # Must have valid immature percentage
        (df['IMMATURE_PERCENT'] > immature_percent_high)  # High immature forms
    )

    # Mark abnormal WBC
    df['abnormal_wbc'] = 0
    df.loc[abnormal_wbc_mask, 'abnormal_wbc'] = 1

    return df


observations = mark_abnormal_temperature(observations)
observations = mark_abnormal_heart_rate(observations)
observations = mark_abnormal_respiratory_rate(observations)
observations = mark_abnormal_wbc(observations)
print("\nAbnormal Temperature Distribution:")
print(observations['abnormal_temperature'].value_counts())
print("\nAbnormal Heart Rate Distribution:")
print(observations['abnormal_heart_rate'].value_counts())
print("\nAbnormal Respiratory Rate Distribution:")
print(observations['abnormal_respiratory_rate'].value_counts())
print("\nAbnormal WBC Distribution:")
print(observations['abnormal_wbc'].value_counts())


Abnormal Temperature Distribution:
abnormal_temperature
0    530730
1       414
Name: count, dtype: int64

Abnormal Heart Rate Distribution:
abnormal_heart_rate
0    527897
1      3247
Name: count, dtype: int64

Abnormal Respiratory Rate Distribution:
abnormal_respiratory_rate
0    523569
1      7575
Name: count, dtype: int64

Abnormal WBC Distribution:
abnormal_wbc
0    526468
1      4676
Name: count, dtype: int64


In [2]:
def get_patients_with_multiple_abnormals_timeframe(observations_df, timeframe_hours=168):
    """
    Get patients with multiple DIFFERENT TYPES of abnormal events within specified timeframe.

    Parameters:
    -----------
    observations_df : pd.DataFrame
        Observations dataframe with abnormal_* columns
    timeframe_hours : int, default 168
        Timeframe in hours to look for multiple abnormal events

    Returns:
    --------
    pd.DataFrame
        Patients with multiple different types of abnormal events within specified timeframe
    """
    df = observations_df.copy()
    df['DATE'] = pd.to_datetime(df['DATE'], errors='coerce')

    # Find abnormal columns
    abnormal_cols = [col for col in df.columns if col.startswith('abnormal_')]

    # Filter abnormal rows
    abnormal_rows = df[df[abnormal_cols].any(axis=1)].copy()

    if len(abnormal_rows) == 0:
        return pd.DataFrame()

    # Sort by patient and date
    abnormal_rows = abnormal_rows.sort_values(['PATIENT', 'DATE'])

    # Group by patient and find events within timeframe
    results = []

    for patient_id, patient_data in abnormal_rows.groupby('PATIENT'):
        if len(patient_data) < 2:  # Need at least 2 events
            continue

        # Check each event against all others within timeframe
        for i, event1 in patient_data.iterrows():
            for j, event2 in patient_data.iterrows():
                if i >= j:  # Skip same event and already checked pairs
                    continue

                # Calculate time difference in hours
                time_diff = abs((event2['DATE'] - event1['DATE']).total_seconds() / 3600)

                if time_diff <= timeframe_hours:
                    # Found multiple events within timeframe
                    start_time = min(event1['DATE'], event2['DATE'])
                    end_time = max(event1['DATE'], event2['DATE'])

                    # Get all events within this timeframe
                    timeframe_events = patient_data[
                        (patient_data['DATE'] >= start_time) &
                        (patient_data['DATE'] <= end_time)
                    ]

                    # Determine which specific abnormal events occurred
                    event_types = []
                    for _, event in timeframe_events.iterrows():
                        if event['abnormal_temperature'] == 1:
                            event_types.append('temp_high')
                        elif event['abnormal_heart_rate'] == 1:
                            event_types.append('hr_high')
                        elif event['abnormal_respiratory_rate'] == 1:
                            event_types.append('resp_high')
                        elif event['abnormal_wbc'] == 1:
                            event_types.append('wbc_high')

                    # Get unique event types
                    unique_event_types = list(set(event_types))

                    # Require at least 2 DIFFERENT types of abnormalities
                    if len(unique_event_types) >= 2:
                        results.append({
                            'PATIENT': patient_id,
                            'TIMEFRAME_START': start_time,
                            'TIMEFRAME_END': end_time,
                            'EVENTS_IN_TIMEFRAME': len(timeframe_events),
                            'TIMEFRAME_HOURS': time_diff,
                            'EVENT_TYPES': ','.join(sorted(unique_event_types)),
                            'EVENT_DETAILS': timeframe_events[['DATE', 'DESCRIPTION'] + abnormal_cols].to_dict('records')
                        })
                        break  # Found timeframe for this event, move to next
            else:
                continue
            break  # Found timeframe, move to next patient

    if not results:
        return pd.DataFrame()

    result_df = pd.DataFrame(results)

    # Remove duplicates (same patient with overlapping timeframes)
    result_df = result_df.drop_duplicates(subset=['PATIENT'], keep='first')

    return result_df

# Usage:
timeframe_hr=24
multiple_timeframe = get_patients_with_multiple_abnormals_timeframe(observations, timeframe_hours=timeframe_hr)
print(f"Patients with multiple abnormal events within {timeframe_hr} hours: {len(multiple_timeframe)}")

if len(multiple_timeframe) > 0:

    # Show detailed events for first patient
    first_patient = multiple_timeframe.iloc[0]
    print(f"\nDetailed events for patient {first_patient['PATIENT']}:")
    for event in first_patient['EVENT_DETAILS']:
        event_type = []
        if event['abnormal_temperature'] == 1:
            event_type.append('temp_high')
        if event['abnormal_heart_rate'] == 1:
            event_type.append('hr_high')
        if event['abnormal_respiratory_rate'] == 1:
            event_type.append('resp_high')
        if event['abnormal_wbc'] == 1:
            event_type.append('wbc_high')
        print(f"  {event['DATE']}: {event['DESCRIPTION']} ({', '.join(event_type) if event_type else 'unknown'})")

Patients with multiple abnormal events within 24 hours: 704

Detailed events for patient 00126cb9-8460-4747-e302-c3609684531e:
  2013-09-07 23:28:56+00:00: Heart rate (hr_high)
  2013-09-07 23:28:56+00:00: Leukocytes [#/volume] in Blood by Automated count (wbc_high)


In [3]:
# save the multiple_timeframe to csv for manual check
multiple_timeframe.to_csv('data/multiple_timeframe.csv', index=False)


In [4]:
def label_patients_with_sepsis_from_abnormals(multiple_timeframe_df, patients_df):
    """
    Label patients with sepsis based on multiple abnormal events timeframe.

    Parameters:
    -----------
    multiple_timeframe_df : pd.DataFrame
        DataFrame from get_patients_with_multiple_abnormals_timeframe function
    patients_df : pd.DataFrame
        Patients dataframe to add sepsis labels to

    Returns:
    --------
    pd.DataFrame
        Patients dataframe with sepsis labels and sepsis dates added
    """
    # Make a copy of patients to avoid modifying original
    patients_with_sepsis = patients_df.copy()

    # Initialize sepsis columns with proper dtypes
    patients_with_sepsis['bluevia_sepsis'] = 0
    patients_with_sepsis['sepsis_date'] = pd.NaT

    # Convert datetime columns to timezone-naive if they have timezone info
    for col in ['TIMEFRAME_START', 'TIMEFRAME_END']:
        if col in multiple_timeframe_df.columns:
            multiple_timeframe_df[col] = pd.to_datetime(
                multiple_timeframe_df[col], errors='coerce'
            ).dt.tz_localize(None)

    # Process each patient in multiple_timeframe
    for _, row in multiple_timeframe_df.iterrows():
        patient_id = row['PATIENT']
        sepsis_date = row['TIMEFRAME_END']  # Date of second abnormal event
        event_types = row['EVENT_TYPES']

        # Find the patient in patients dataframe and update sepsis info
        patient_mask = patients_with_sepsis['Id'] == patient_id
        patients_with_sepsis.loc[patient_mask, 'bluevia_sepsis'] = 1
        patients_with_sepsis.loc[patient_mask, 'sepsis_date'] = sepsis_date

    # Add additional sepsis details
    # Create a mapping of patient to their sepsis details
    sepsis_details = multiple_timeframe_df.set_index('PATIENT')[
        ['TIMEFRAME_START', 'TIMEFRAME_END', 'EVENTS_IN_TIMEFRAME',
         'TIMEFRAME_HOURS', 'EVENT_TYPES']
    ].to_dict('index')

    # Add detail columns
    patients_with_sepsis['sepsis_timeframe_start'] = patients_with_sepsis['Id'].map(
        lambda x: sepsis_details.get(x, {}).get('TIMEFRAME_START', pd.NaT)
    )
    patients_with_sepsis['sepsis_timeframe_end'] = patients_with_sepsis['Id'].map(
        lambda x: sepsis_details.get(x, {}).get('TIMEFRAME_END', pd.NaT)
    )
    patients_with_sepsis['sepsis_event_types'] = patients_with_sepsis['Id'].map(
        lambda x: sepsis_details.get(x, {}).get('EVENT_TYPES', '')
    )
    patients_with_sepsis['sepsis_events_count'] = patients_with_sepsis['Id'].map(
        lambda x: sepsis_details.get(x, {}).get('EVENTS_IN_TIMEFRAME', 0)
    )
    patients_with_sepsis['sepsis_timeframe_hours'] = patients_with_sepsis['Id'].map(
        lambda x: sepsis_details.get(x, {}).get('TIMEFRAME_HOURS', 0)
    )

    return patients_with_sepsis

# Usage:
# First get multiple abnormal events
timeframe_hr = 24
multiple_timeframe = get_patients_with_multiple_abnormals_timeframe(observations, timeframe_hours=timeframe_hr)
print(f"Patients with multiple different abnormal events within {timeframe_hr} hours: {len(multiple_timeframe)}")

# Then label patients with sepsis
patients_with_sepsis_labels = label_patients_with_sepsis_from_abnormals(multiple_timeframe, patients)

# Verify the labeling
print(f"\nTotal patients: {len(patients_with_sepsis_labels)}")
print(f"Patients with Bluevia sepsis: {patients_with_sepsis_labels['bluevia_sepsis'].sum()}")

# Show sample of labeled patients
print("\nSample of labeled patients:")
print(patients_with_sepsis_labels[[
    'Id', 'FIRST', 'LAST', 'bluevia_sepsis', 'sepsis_date',
    'sepsis_event_types', 'sepsis_events_count', 'sepsis_timeframe_hours'
]].head(10))

# Show distribution
print("\nBluevia sepsis distribution:")
print(patients_with_sepsis_labels['bluevia_sepsis'].value_counts())

# Save updated patients dataframe
patients_with_sepsis_labels.to_csv('data/patients_with_bluevia_sepsis.csv', index=False)

Patients with multiple different abnormal events within 24 hours: 704

Total patients: 1163
Patients with Bluevia sepsis: 704

Sample of labeled patients:
                                     Id       FIRST            LAST  \
0  b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85    Damon455      Langosh790   
1  c1f1fcaa-82fd-d5b7-3544-c8f9708b06a8       Thi53       Wunsch504   
2  339144f8-50e1-633e-a013-f361391c4cff      Chi716  Greenfelder433   
3  d488232e-bf14-4bed-08c0-a82f34b6a197  Phillis443       Walter473   
4  217f95a3-4e10-bd5d-fb67-0cfb5e8ba075  Jerrold404       Herzog843   
5  faac724a-a9e9-be66-fe1e-3044dc0ba8ea  Brandon214      Watsica258   
6  23d16ee3-8cd4-eeb8-e77e-1e5fbf4c4159  Rodrigo242       Arellano2   
7  aade3c61-92bd-d079-9d28-0b2b7fde0fbb      Sal878        Hoppe518   
8  0288c42c-43a1-9878-4a9d-6b96caa12c40     Miki234        Kozey370   
9  dc6c06d0-a7d8-100f-c08b-46c93700c188    Taylor21    Gulgowski816   

   bluevia_sepsis         sepsis_date           sepsis_event_ty